# COVID Hospitalization Weekly Scrapper

This script is intended for obtaining the data from the OHSA Tableau dashboard for COVID hospitalization; as well as reformatting the data to then append into the data record.

In [1]:
#import libraries and packages
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
#Remember to update link below with latest download link from https://public.tableau.com/app/profile/oregon.health.authority.covid.19/viz/OregonCOVID-19HospitalCapacitySummaryTables_15965754787060/HospitalizationsByFacility
downloaded_data = pd.read_csv('https://public.tableau.com/vizql/w/OregonCOVID-19HospitalCapacitySummaryTables_15965754787060/v/HospitalizationsByFacility/tempfile/sessions/300C3B28D8CC4165BA2ED421F58A214C-0:0/?key=1798539847&keepfile=yes&attachment=yes',encoding='utf-16le',sep='\t',skiprows=1)

In [3]:
downloaded_data

,Hospital Name,County,Region,10,11,13,20,<10
0,Adventist Health Portland,Multnomah,Region 1,NaN,NaN,NaN,NaN,NaN
1,Adventist Health Tillamook,Tillamook,Region 1,NaN,NaN,NaN,NaN,NaN
2,Asante Ashland Community Hospital,Jackson,Region 5,NaN,NaN,NaN,NaN,NaN
3,Asante Rogue Regional Medical Center,Jackson,Region 5,NaN,NaN,NaN,NaN,NaN
4,Asante Three Rivers Medical Center,Josephine,Region 5,NaN,NaN,NaN,NaN,NaN
5,Bay Area Hospital,Coos,Region 3,NaN,NaN,NaN,NaN,NaN
6,Columbia Memorial Hospital,Clatsop,Region 1,NaN,NaN,NaN,NaN,NaN
7,Curry General Hospital,Curry,Region 3,NaN,NaN,NaN,NaN,NaN
8,Good Samaritan Regional Medical Center,Benton,Region 2,NaN,NaN,NaN,NaN,NaN
9,Good Shepherd Health Care System,Umatilla,Region 9,NaN,NaN,NaN,NaN,NaN


In [4]:
# Define the columns to exclude from the search for hospitalization data
excluded_columns = ['Hospital Name', 'County', 'Region', '<10']

# Identify columns to search for hospitalization data
hospitalization_columns = [col for col in downloaded_data.columns if col not in excluded_columns]

In [5]:
# Function to apply across each row
def get_hospitalization_value(row):
    # Iterate over the hospitalization columns
    for col in hospitalization_columns:
        # Check if the value is not NaN (using pd.notna because np.nan != np.nan)
        if pd.notna(row[col]):
            return row[col]
    # Default value if all relevant columns are NaN
    return '<10'

In [6]:
# Create the new column by applying the function
downloaded_data['Hospitalization Cases'] = downloaded_data.apply(get_hospitalization_value, axis=1)

#Drop columns that are no longer needed
downloaded_data.drop(hospitalization_columns + ['<10'], axis=1, inplace=True)

In [7]:
downloaded_data['MAX_7 Day Average of COVID-19 Positive Patients_Num']=downloaded_data['Hospitalization Cases'].replace('<10', '9').astype(int)
downloaded_data['MIN_7 Day Average of COVID-19 Positive Patients_Num']=downloaded_data['Hospitalization Cases'].replace('<10', '1').astype(int)

In [8]:
# Group by 'County' and sum both the max and min columns
aggregated_data = downloaded_data.groupby('County').agg({
    'MAX_7 Day Average of COVID-19 Positive Patients_Num': 'sum',
    'MIN_7 Day Average of COVID-19 Positive Patients_Num': 'sum'
}).reset_index()

In [9]:
# Function to prompt the user for a date and validate the format
def get_user_date_input(prompt):
    while True:
        user_input = input(prompt)
        try:
            # Attempt to convert the user input into a date using the specified format
            valid_date = datetime.datetime.strptime(user_input, '%m/%d/%Y')
            # If conversion is successful, return the formatted date
            return valid_date.strftime('%m/%d/%Y')
        except ValueError:
            # If conversion fails, notify the user and prompt again
            print("The date format should be m/d/YYYY. Please try again.")

In [10]:
# Prompt the user to enter the date
date_input = get_user_date_input("Enter the date when the Tableau dashboard was last updated (m/d/YYYY): ")

print(f"The entered date is: {date_input}")


The entered date is: 02/20/2024


In [11]:
aggregated_data['DateStamp']=date_input

In [12]:
# Pivot the DataFrame
df_pivot = aggregated_data.pivot(index='DateStamp', columns='County', values=['MAX_7 Day Average of COVID-19 Positive Patients_Num', 'MIN_7 Day Average of COVID-19 Positive Patients_Num'])

# Adjust column levels
df_pivot.columns = df_pivot.columns.swaplevel(0, 1)
df_pivot.sort_index(axis=1, level=0, inplace=True)

In [13]:
df_pivot

County                                                  Benton  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  9    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  1    

County                                               Clackamas  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                 36    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  4    

County                                                 Clatsop  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                 18    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  2    

County                                                    Coos  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  9    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  1    

County                                                   Crook  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  9    

County                                                          ...  \
           MIN_7 Day Average of COVID-19 Positive Patients_Num  ...   
DateStamp                                                       ...   
02/20/2024                                                  1   ...   

County                                                   Union  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  9    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  1    

County                                                 Wallowa  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  9    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  1    

County                                                   Wasco  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
DateStamp                                                        
02/20/2024                                                  9    

Count

## Read Existing File and Append New Data

In [14]:
# change directory folder
os.chdir('Z:/')
os.getcwd() #to get working directory

'Z:\\'

In [15]:
# Read the existing Excel file
existing_data = pd.read_excel("COVID_Hospitalized.xlsx",header=[0, 1], index_col=0)


In [16]:
#Align column order
df_pivot=df_pivot.reindex(columns=existing_data.columns)

In [18]:
# Append the data
combined_data = existing_data.append(df_pivot, ignore_index=False)

<ipython-input-18-ae525f5ced4f>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = existing_data.append(df_pivot, ignore_index=False)


In [19]:
# Replace NaN values with 0
combined_data.fillna(0, inplace=True)

In [20]:
# Convert all columns to integers, except for the excluded (non-numeric) ones
# Identify non-numeric columns first (if any)
non_numeric_columns = combined_data.select_dtypes(include=['object']).columns

# Convert numeric columns to integers
for column in combined_data.columns.difference(non_numeric_columns):
    combined_data[column] = combined_data[column].astype(int)

In [21]:
combined_data.tail()

County                                                   Baker  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                   9    
1/30/2024                                                   9    
2/6/2024                                                    9    
02/13/2024                                                  9    
02/20/2024                                                  0    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                   1    
1/30/2024                                                   1    
2/6/2024                                                    1    
02/13/2024                                                  1    
02/20/2024                                                  0    

County                                                  Benton  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                   9    
1/30/2024                                                   9    
2/6/2024                                                    9    
02/13/2024                                                  9    
02/20/2024                                                  9    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                   1    
1/30/2024                                                   1    
2/6/2024                                                    1    
02/13/2024                                                  1    
02/20/2024                                                  1    

County                                               Clackamas  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                  37    
1/30/2024                                                  39    
2/6/2024                                                   36    
02/13/2024                                                 36    
02/20/2024                                                 36    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                  13    
1/30/2024                                                  15    
2/6/2024                                                    4    
02/13/2024                                                  4    
02/20/2024                                                  4    

County                                                 Clatsop  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                   9    
1/30/2024                                                  18    
2/6/2024                                                    9    
02/13/2024                                                  0    
02/20/2024                                                 18    

County                                                          \
           MIN_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                   1    
1/30/2024                                                   2    
2/6/2024                                                    1    
02/13/2024                                                  0    
02/20/2024                                                  2    

County                                                    Coos  \
           MAX_7 Day Average of COVID-19 Positive Patients_Num   
1/23/2024                                                   9    
1/30/2024                                                   9    
2/6/2024                        

In [22]:
# Save the updated DataFrame back to the Excel file
combined_data.to_excel("COVID_Hospitalized.xlsx")